<a href="https://colab.research.google.com/github/jlmcnamara/GettingandCleaningDataCourseProject/blob/master/Python_Script_Newsletter_Bullet_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

# --- Configuration & Brand-Specific Rules ---

# Thresholds for adding "partially offset by..." note on drivers
# Key: Company Name (lowercase), Value: Growth percentage threshold (e.g., 5 for < +5%)
PARTIALLY_OFFSET_THRESHOLD = {
    "jakks pacific": 5,
    "mattel": 10,
    # Add other companies here if needed
    "default": 10 # Default threshold if company not specified
}

# --- Helper Functions ---

def format_currency(value_millions):
    """Formats a number in millions into $XM or $XB format."""
    if value_millions is None:
        return "{Value}" # Placeholder if no value provided
    # Round to nearest million first
    rounded_value = round(value_millions)
    if rounded_value >= 1000:
        return f"${rounded_value/1000:.1f}B".replace(".0B", "B") # Format as billions
    else:
        return f"${rounded_value}M" # Format as millions

def format_comparison(percentage):
    """Formats the comparison percentage string (+X% / -X% / flat vs PY)."""
    if percentage is None:
        return "{Comparison}" # Placeholder
    if percentage == 0:
        return "(flat vs PY)"
    # No leading zero for single digits, include sign
    sign = "+" if percentage > 0 else "" # Negative sign is already part of the number
    return f"({sign}{percentage}% vs PY)"

def get_offset_condition_text(company_name, comparison_percentage):
    """Gets the conditional text reminder for 'partially offset by'."""
    if comparison_percentage is None:
        return ""

    threshold = PARTIALLY_OFFSET_THRESHOLD.get(company_name.lower(), PARTIALLY_OFFSET_THRESHOLD["default"])

    # Note: The logic is applied *if growth is LESS THAN the threshold*
    # Based on user request: "if growth < +5%"
    if comparison_percentage < threshold:
        return f' - consider "partially offset by..." if growth < +{threshold}%'
    # Handle the specific case for Mattel Q1 ER (+12%) which is >= +10%
    # Based on original examples, strong double digit growth *also* sometimes had this note.
    # Let's refine this: Add note if growth is < threshold OR >= 10% (strong growth often has offsets)
    elif comparison_percentage >= 10:
         return f' - consider "partially offset by..."' # Simplified note for strong growth
    else:
        return "" # No note needed if growth is >= threshold but < 10%

def get_mirroring_condition_text(partner_comparison_percentage, dcp_comparison_percentage):
    """Gets the conditional text reminder for 'mirroring performance'."""
    if partner_comparison_percentage is None or dcp_comparison_percentage is None:
        return ""
    # Check if DCP % is within 1 basis point (inclusive)
    if abs(partner_comparison_percentage - dcp_comparison_percentage) <= 1:
        return '. {If DCP % is within +/-1% of partner\'s overall %, consider adding phrase like "mirroring partner performance"}'
    else:
        return ""

# --- Main Generation Function ---

def generate_bullets(data):
    """
    Generates the Markdown bullet points for a given company's data.

    Args:
        data (dict): A dictionary containing the data for one company.
                     Expected keys: company_name, headline, headline_url,
                                    source, box_url, time_period,
                                    partner_revenue_m, partner_comparison_pct,
                                    partner_expectations, partner_drivers_placeholder,
                                    partner_outlook_placeholder,
                                    dcp_er_q_m, dcp_comparison_q_pct,
                                    dcp_drivers_q_placeholder,
                                    dcp_er_fy_m, dcp_comparison_fy_pct,
                                    dcp_drivers_fy_placeholder,
                                    dcp_collabs_placeholder

    Returns:
        str: A string containing the formatted Markdown bullets.
    """
    company_name = data.get("company_name", "{Company Name}")
    offset_note_threshold = PARTIALLY_OFFSET_THRESHOLD.get(company_name.lower(), PARTIALLY_OFFSET_THRESHOLD["default"])

    # Format values using helper functions
    partner_revenue_formatted = format_currency(data.get("partner_revenue_m"))
    partner_comparison_formatted = format_comparison(data.get("partner_comparison_pct"))
    dcp_er_q_formatted = format_currency(data.get("dcp_er_q_m"))
    dcp_comparison_q_formatted = format_comparison(data.get("dcp_comparison_q_pct"))
    dcp_er_fy_formatted = format_currency(data.get("dcp_er_fy_m"))
    dcp_comparison_fy_formatted = format_comparison(data.get("dcp_comparison_fy_pct"))

    # Get conditional notes
    partner_offset_note = get_offset_condition_text(company_name, data.get("partner_comparison_pct"))
    dcp_q_offset_note = get_offset_condition_text(company_name, data.get("dcp_comparison_q_pct"))
    dcp_fy_offset_note = get_offset_condition_text(company_name, data.get("dcp_comparison_fy_pct"))
    mirroring_note = get_mirroring_condition_text(data.get("partner_comparison_pct"), data.get("dcp_comparison_q_pct"))

    # Handle negative FY comparison phrasing for drivers (lead with negative)
    dcp_drivers_fy_placeholder = data.get("dcp_drivers_fy_placeholder", "{Explanation/Drivers for outlook}")
    if data.get("dcp_comparison_fy_pct", 0) < 0:
         # Add a reminder note if the default placeholder is still there and result is negative
         if "{Explanation/Drivers for outlook}" in dcp_drivers_fy_placeholder or "expected H2 softness" in dcp_drivers_fy_placeholder: # Check for default or known negative placeholder
              dcp_drivers_fy_placeholder = "{Lead with negative driver, e.g., reflects expected H2 softness, partially offset by strong H1}"
         # Otherwise, assume user provided appropriate negative-leading explanation

    # Build the Markdown string
    output = f"## {company_name} Placeholder\n\n" # Use company name in title
    output += f"`[{company_name} {data.get('headline', '{Headline}')}]({data.get('headline_url', '{HeadlineURL}')}) ({data.get('source', '{Source}')}) ([PDF]({data.get('box_url', '{BoxURL}')}))`\n"

    # Bullet 1: Partner Overall
    output += (f"o   {data.get('time_period', 'Jan-Mar')} revenue {partner_revenue_formatted} {partner_comparison_formatted}. "
               f"Revenue {data.get('partner_expectations', '{beat/met/missed}')} expectations, "
               f"driven by {data.get('partner_drivers_placeholder', '{Explanation/Drivers}')}{partner_offset_note}. "
               f"{data.get('partner_outlook_placeholder', '{Partner\\'s outlook explanation}')}\n")

    # Bullet 2: DCP Past Quarter
    output += (f"o   DCP {data.get('time_period', 'Jan-Mar')} ER {dcp_er_q_formatted} {dcp_comparison_q_formatted} "
               f"primarily due to {data.get('dcp_drivers_q_placeholder', '{Explanation mentioning specific franchises/products driving Q1 performance}')}{dcp_q_offset_note}"
               f"{mirroring_note}\n")

    # Bullet 3: DCP Future FY
    output += (f"o   DCP FY25 ER {dcp_er_fy_formatted} {dcp_comparison_fy_formatted} "
               f"reflects {dcp_drivers_fy_placeholder}{dcp_fy_offset_note}. "
               f"{data.get('dcp_collabs_placeholder', '{Include present/future collaborations}')}\n")

    return output

# --- Example Usage ---

if __name__ == "__main__":
    # Example Data Input (replace with actual data gathering)
    jakks_data = {
        "company_name": "JAKKS Pacific",
        "headline": "{Headline}", # Placeholder
        "headline_url": "{HeadlineURL}", # Placeholder
        "source": "{Source}", # Placeholder
        "box_url": "{BoxURL}", # Placeholder
        "time_period": "Jan-Mar",
        "partner_revenue_m": None, # Placeholder - No data yet
        "partner_comparison_pct": None, # Placeholder - No data yet
        "partner_expectations": "{beat/met/missed}", # Placeholder
        "partner_drivers_placeholder": "{Explanation/Drivers}", # Placeholder
        "partner_outlook_placeholder": "{Partner's outlook explanation}", # Placeholder
        "dcp_er_q_m": 11.8, # Actual: 11.8 -> Rounds to 12M
        "dcp_comparison_q_pct": 5, # Actual: +5%
        "dcp_drivers_q_placeholder": "{Explanation mentioning specific franchises/products driving Q1 performance}", # Placeholder
        "dcp_er_fy_m": 56.4, # Actual: 56.4 -> Rounds to 56M
        "dcp_comparison_fy_pct": 0, # Actual: 0% -> flat
        "dcp_drivers_fy_placeholder": "stable performance in HL driven by XX and XX", # From user adjustment
        "dcp_collabs_placeholder": "{Include present/future collaborations}" # From user adjustment
    }

    mattel_data = {
        "company_name": "Mattel",
        "headline": "{Headline}", # Placeholder
        "headline_url": "{HeadlineURL}", # Placeholder
        "source": "{Source}", # Placeholder
        "box_url": "{BoxURL}", # Placeholder
        "time_period": "Jan-Mar",
        "partner_revenue_m": None, # Placeholder - No data yet
        "partner_comparison_pct": None, # Placeholder - No data yet
        "partner_expectations": "{beat/met/missed}", # Placeholder
        "partner_drivers_placeholder": "{Explanation/Drivers}", # Placeholder
        "partner_outlook_placeholder": "{Partner's outlook explanation}", # Placeholder
        "dcp_er_q_m": 26.0, # Actual: 26.0 -> 26M
        "dcp_comparison_q_pct": 12, # Actual: +12%
        "dcp_drivers_q_placeholder": "{Explanation mentioning specific franchises/products driving Q1 performance}", # Placeholder
        "dcp_er_fy_m": 76.3, # Actual: 76.3 -> 76M
        "dcp_comparison_fy_pct": -1, # Actual: -1%
        "dcp_drivers_fy_placeholder": "{expected H2 softness, partially offset by strong H1}", # Default negative placeholder
        "dcp_collabs_placeholder": "{Include present/future collaborations}" # From user adjustment
    }

    # Generate and print bullets
    print(generate_bullets(jakks_data))
    print("---") # Separator
    print(generate_bullets(mattel_data))